In [1]:
import ee 
from ee_jupyter.ipyleaflet import Map
import ipyleaflet as ipl
from ipywidgets import interact, interactive, fixed, interact_manual, Layout

ee.Initialize()

Some Initialization.

In [2]:
clouds=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD")
precipitation=ee.ImageCollection("NASA/GPM_L3/IMERG_V06")
nox=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")
sox=ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_SO2")

IC=[clouds,precipitation,nox,sox]
bands=[
    'cloud_optical_depth',
    'precipitationCal',
    'NO2_column_number_density',
    'SO2_column_number_density'
]

import json

with open(r'../shp/china.json','r') as f:
    china = json.load(f)
china_fc = ee.FeatureCollection(china)
china_img = ee.Image().paint(china_fc, 0, 1)

with open(r'../shp/Beijing.json','r') as f:
    beijing = json.load(f)
beijing_fc = ee.FeatureCollection(beijing)
beijing_img = ee.Image().paint(beijing_fc, 0, 1)

start_year = 2019
end_year = 2022

下面的Cell是关键函数Cell, 请认真对待和调试!

In [4]:
def monthly_mean(image_collection,region,band):
    months = ee.List.sequence(1, 12)
    years = ee.List.sequence(start_year, end_year)
    def by_year(y):
        def by_month(m):
            return image_collection.filter(ee.Filter.calendarRange(y, y, 'year')) \
                .filter(ee.Filter.calendarRange(m, m, 'month')) \
                .select(band).mean().clip(region) \
                .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
        return months.map(by_month)
    return ee.ImageCollection.fromImages(years.map(by_year).flatten())


### 1. monthly_mean函数的调试

In [5]:
clouds_monthly_mean=monthly_mean(clouds,beijing_fc,bands[0])
size = clouds_monthly_mean.size().getInfo()
size

48

In [6]:
first_img=clouds_monthly_mean.first()
first_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'cloud_optical_depth',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:time_start': 1546300800000, 'system:index': '0'}}

In [7]:
map_0 = Map(center=[39.9, 116.4], zoom=8, layout=Layout(height='600px'), basemap=ipl.basemaps.Esri.WorldImagery)
map_0.addLayer(first_img,{'min':1,'max':50,'palette':['blue','skyblue','white']},'cloud_optical_depth')
map_0.addLayer(beijing_img,{'palette':'grey'},'Beijing')
map_0

Map(center=[39.9, 116.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

### 2. cross_correlation函数的测试

In [22]:
def cross_correlation(image_collection1, image_collection2, band1, band2, lag_range):
    def by_lag(l):
        # 将第二个影像集按照滞后值 l 进行时间平移
        def shift_time(image):
            return image.set('system:time_start', ee.Date(image.get('system:time_start')).advance(l, 'month').millis())

        shifted = image_collection2.map(shift_time)

        # 创建一个基于时间的联接条件
        filter = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')

        # 使用 saveFirst 联接方法
        join = ee.Join.saveFirst('shifted')
        combined = join.apply(image_collection1, shifted, filter)

        # 对结合后的影像计算乘积并求和，获取对应 lag 的 image
        def calc_product(image):
            correlation = ee.Image(image).select([band1]).multiply(ee.Image(image.get('shifted')).select([band2]))
            return ee.Image().addBands(correlation.rename('correlation'))

        return ee.ImageCollection(combined).map(calc_product).sum().set('lag', l)

    def max_correlation_and_lag(lag_images):
        # 初始化一个影像，一个波段用于存储最大交叉相关性值，一个波段用于存储对应的滞后值
        max_corr = ee.Image.constant(-1e5).select([0], ['max'])  # 假设的最小值
        max_lag = ee.Image.constant(0).select([0], ['lag'])

        # 定义一个更新函数，用于更新最大值和滞后值
        def update_max(image, prev):
            current = ee.Image(prev)
            new_max = image.select('correlation')
            new_lag = ee.Image.constant(image.get('lag')).select([0], ['lag'])
            condition = new_max.gt(current.select('max'))
            return current.where(condition, ee.Image.cat(new_max, new_lag))

        # 对于每个滞后值的影像，更新最大交叉相关性值及其滞后值
        combined_max_lag = lag_images.iterate(update_max, ee.Image.cat(max_corr, max_lag))

        return ee.Image(combined_max_lag)

    # 应用每个滞后值
    lags = ee.List.sequence(-lag_range, lag_range)
    lag_images = ee.ImageCollection(lags.map(by_lag))

    # 获取每个像素的最大交叉相关性及其对应的滞后值
    max_corr_and_lag_image = max_correlation_and_lag(lag_images)

    return max_corr_and_lag_image

# 示例调用（假设有可用的image_collection1和image_collection2）
# max_corr_and_lag = cross_correlation(image_collection1, image_collection2, 'band1', 'band2', 12)


In [8]:
precipitation_monthly_mean=monthly_mean(precipitation,beijing_fc,bands[1])
print(precipitation_monthly_mean.size().getInfo())
precipitation_monthly_mean.getInfo()

48


{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'precipitationCal',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1546300800000, 'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitationCal',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1548979200000, 'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitationCal',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:time_start': 1551398400000, 'system:index': '2'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitationCal',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPS

In [23]:
import socks
import socket
socks.set_default_proxy(socks.PROXY_TYPE_HTTP, "127.0.0.1", 7890)
socket.socket = socks.socksocket

def resample_collection(image_collection, scale, projection):
    def resample_image(image):
        return image.resample('bilinear').reproject(projection.atScale(scale))

    return image_collection.map(resample_image)

clouds_monthly_mean=resample_collection(clouds_monthly_mean,1000,beijing_img.projection())
precipitation_monthly_mean=resample_collection(precipitation_monthly_mean,1000,beijing_img.projection())
cloudsXprecipitation=cross_correlation(clouds_monthly_mean,precipitation_monthly_mean,bands[0],bands[1],48)

import time
def export_image(image,region,scale):
    task=ee.batch.Export.image.toDrive(
        image=image,
        description='cross',
        folder='GEE',
        region=region.geometry(),
        scale=scale,
        crs='EPSG:4326'
    )
    task.start()
    i=0
    while task.active():
        print(f'Running for {i*5} seconds')
        time.sleep(5)
        i+=1
    if task.status()['state']=='COMPLETED':
        print('Exporting completed')
    else:
        print('Something went wrong')

export_image(cloudsXprecipitation,beijing_fc,1000)

Running for 0 seconds
Running for 5 seconds
Running for 10 seconds
Running for 15 seconds
Running for 20 seconds
Running for 25 seconds
Running for 30 seconds
Running for 35 seconds
Running for 40 seconds
Something went wrong


In [27]:
def by_lag(l,image_collection1, image_collection2, band1, band2):
        # 将第二个影像集按照滞后值 l 进行时间平移
        def shift_time(image):
            return image.set('system:time_start', ee.Date(image.get('system:time_start')).advance(l, 'month').millis())

        shifted = image_collection2.map(shift_time)

        # 创建一个基于时间的联接条件
        filter = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')

        # 使用 saveFirst 联接方法
        join = ee.Join.saveFirst('shifted')
        combined = join.apply(image_collection1, shifted, filter)

        # 对结合后的影像计算乘积并求和，获取对应 lag 的 image
        def calc_product(image):
            # 转换为浮点型
            img1 = ee.Image(image).select([band1]).toDouble()
            img2 = ee.Image(image.get('shifted')).select([band2]).toDouble()
            
            # 计算交叉相关性
            correlation = img1.multiply(img2)
            
            return ee.Image().addBands(correlation.rename('correlation'))

        return ee.ImageCollection(combined).map(calc_product).sum().set('lag', l)

lag=by_lag(2,clouds_monthly_mean,precipitation_monthly_mean,bands[0],bands[1])
export_image(lag,beijing_fc,1000)

Running for 0 seconds
Running for 5 seconds
Running for 10 seconds
Running for 15 seconds
Running for 20 seconds
Running for 25 seconds
Running for 30 seconds
Running for 35 seconds
Running for 40 seconds
Running for 45 seconds
Running for 50 seconds
Running for 55 seconds
Running for 60 seconds
Running for 65 seconds
Running for 70 seconds
Something went wrong
